# Tabular EDA

Import packages:

In [ ]:
!pip install ydata-profiling

In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from IPython.display import IFrame

#Style configuration
pd.set_option('display.max_columns', 1000)

Relevant paths:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
dset_path = '/content/drive/MyDrive/MLDS1/02. Entendimiento del negocio y análisis exploratorio/EDA/data/clinical_data.csv'

# 1. Clinical exploration

In [9]:
clinal_dset = pd.read_csv(dset_path)
clinal_dset.head()

,tcga_bcr,admin,bcr,file_uuid,batch_number,project_code,disease_code,day_of_dcc_upload,month_of_dcc_upload,year_of_dcc_upload,patient_withdrawal,withdrawn,program,dbgap_registration_code,patient,additional_studies,tumor_tissue_site,tumor_tissue_site_other,other_dx,gender,vital_status,days_to_birth,days_to_last_known_alive,days_to_death,days_to_last_followup,race_list,race,bcr_patient_barcode,tissue_source_site,patient_id,bcr_patient_uuid,history_of_neoadjuvant_treatment,informed_consent_verified,icd_o_3_site,icd_o_3_histology,icd_10,days_to_initial_pathologic_diagnosis,age_at_initial_pathologic_diagnosis,year_of_initial_pathologic_diagnosis,ethnicity,person_neoplasm_cancer_status,primary_lymph_node_presentation_assessment,lymph_node_examined_count,er_detection_method_text,pgr_detection_method_text,anatomic_neoplasm_subdivisions,anatomic_neoplasm_subdivision,her2_neu_chromosone_17_signal_ratio_value,axillary_lymph_node_stage_method_type,axillary_lymph_node_stage_other_method_descriptive_text,breast_carcinoma_surgical_procedure_name,breast_neoplasm_other_surgical_procedure_descriptive_text,breast_carcinoma_primary_surgical_procedure_name,surgical_procedure_purpose_other_text,histological_type,histological_type_other,menopause_status,breast_carcinoma_progesterone_receptor_status,cytokeratin_immunohistochemistry_staining_method_micrometastasis_indicator,breast_carcinoma_immunohistochemistry_er_pos_finding_scale,immunohistochemistry_positive_cell_score,her2_immunohistochemistry_level_result,breast_cancer_surgery_margin_status,margin_status,initial_pathologic_diagnosis_method,init_pathology_dx_method_other,lab_procedure_her2_neu_in_situ_hybrid_outcome_type,breast_carcinoma_estrogen_receptor_status,lab_proc_her2_neu_immunohistochemistry_receptor_status,number_of_lymphnodes_positive_by_ihc,number_of_lymphnodes_positive_by_he,pos_finding_progesterone_receptor_other_measurement_scale_text,positive_finding_estrogen_receptor_other_measurement_scale_text,her2_erbb_pos_finding_cell_percent_category,pos_finding_her2_erbb2_other_measurement_scale_text,her2_erbb_method_calculation_method_text,her2_neu_and_centromere_17_copy_number_analysis_input_total_number_count,her2_and_centromere_17_positive_finding_other_measurement_scale_text,her2_erbb_pos_finding_fluorescence_in_situ_hybridization_calculation_method_text,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,fluorescence_in_situ_hybridization_diagnostic_procedure_chromosome_17_signal_result_range,first_nonlymph_node_metastasis_anatomic_sites,metastatic_site_at_diagnosis,metastatic_site_at_diagnosis_other,er_level_cell_percentage_category,progesterone_receptor_level_cell_percent_category,distant_metastasis_present_ind2,metastatic_breast_carcinoma_estrogen_receptor_status,metastatic_breast_carcinoma_estrogen_receptor_level_cell_percent_category,metastatic_breast_carcinoma_immunohistochemistry_er_pos_cell_score,pos_finding_metastatic_breast_carcinoma_estrogen_receptor_other_measuremenet_scale_text,metastatic_breast_carcinoma_estrogen_receptor_detection_method_text,metastatic_breast_carcinoma_progesterone_receptor_status,metastatic_breast_carcinoma_lab_proc_her2_neu_immunohistochemistry_receptor_status,metastatic_breast_carcinoma_progesterone_receptor_level_cell_percent_category,metastatic_breast_carcinoma_immunohistochemistry_pr_pos_cell_score,metastatic_breast_carcinoma_pos_finding_progesterone_receptor_other_measure_scale_text,metastatic_breast_carcinoma_progesterone_receptor_detection_method_text,metastatic_breast_carcinoma_her2_erbb_pos_finding_cell_percent_category,metastatic_breast_carcinoma_erbb2_immunohistochemistry_level_result,metastatic_breast_carcinoma_pos_finding_her2_erbb2_other_measure_scale_text,metastatic_breast_carcinoma_her2_erbb_method_calculation_method_text,metastatic_breast_carcinoma_lab_proc_her2_neu_in_situ_hybridization_outcome_type,metastatic_breast_carcinoma_fluorescence_in_situ_hybridization_diagnostic_proc_centromere_17_signal_result

## 1.1 Dataset correction

In [10]:
corrected_clinical_dset = pd.DataFrame()
columns_name = list(clinal_dset.columns)
cleaned_variables = []

for column_name in columns_name:
    column = clinal_dset[column_name]

    #Check unique values
    unique_values = column.unique()
    unique_values = unique_values[np.invert(pd.isna(unique_values))]
    unique_coverage = len(unique_values)/len(column)

    #Count nulls
    nan_counts = column.isna().sum()
    coverage = (len(column) - nan_counts)/len(column)

    if not((len(unique_values) <= 1) or (unique_coverage == 1) or (coverage < 0.1)):

        #Object columns cleaning
        column = column.map(lambda value : value.replace('\n', ' ') if type(value)==str else value)

        corrected_clinical_dset = pd.concat([corrected_clinical_dset, column], axis = 1)
    else:
        cleaned_variables.append(column_name)

corrected_clinical_dset.head()


,batch_number,disease_code,other_dx,gender,vital_status,days_to_birth,days_to_death,days_to_last_followup,race_list,race,tissue_source_site,history_of_neoadjuvant_treatment,icd_o_3_site,icd_o_3_histology,icd_10,age_at_initial_pathologic_diagnosis,year_of_initial_pathologic_diagnosis,ethnicity,person_neoplasm_cancer_status,primary_lymph_node_presentation_assessment,lymph_node_examined_count,er_detection_method_text,pgr_detection_method_text,anatomic_neoplasm_subdivisions,anatomic_neoplasm_subdivision,her2_neu_chromosone_17_signal_ratio_value,axillary_lymph_node_stage_method_type,breast_carcinoma_surgical_procedure_name,surgical_procedure_purpose_other_text,histological_type,menopause_status,breast_carcinoma_progesterone_receptor_status,cytokeratin_immunohistochemistry_staining_method_micrometastasis_indicator,breast_carcinoma_immunohistochemistry_er_pos_finding_scale,immunohistochemistry_positive_cell_score,her2_immunohistochemistry_level_result,margin_status,initial_pathologic_diagnosis_method,lab_procedure_her2_neu_in_situ_hybrid_outcome_type,breast_carcinoma_estrogen_receptor_status,lab_proc_her2_neu_immunohistochemistry_receptor_status,number_of_lymphnodes_positive_by_ihc,number_of_lymphnodes_positive_by_he,pos_finding_progesterone_receptor_other_measurement_scale_text,positive_finding_estrogen_receptor_other_measurement_scale_text,her2_erbb_pos_finding_cell_percent_category,tissue_prospective_collection_indicator,tissue_retrospective_collection_indicator,first_nonlymph_node_metastasis_anatomic_sites,er_level_cell_percentage_category,progesterone_receptor_level_cell_percent_category,distant_metastasis_present_ind2,breast_carcinoma_immunohistochemistry_progesterone_receptor_pos_finding_scale,breast_carcinoma_immunohistochemistry_pos_cell_score,stage_event,system_version,pathologic_stage,tnm_categories,pathologic_categories,pathologic_T,pathologic_N,pathologic_M,postoperative_rx_tx,radiation_therapy,new_tumor_events,new_tumor_event_after_initial_treatment,day_of_form_completion,month_of_form_completion,year_of_form_completion,follow_ups,follow_up,bcr_followup_barcode,bcr_followup_uuid,followup_case_report_form_submission_reason,lost_follow_up,new_tumor_event,drugs,drug,tx_on_clinical_trial,regimen_number,bcr_drug_barcode,bcr_drug_uuid,total_dose,total_dose_units,prescribed_dose,prescribed_dose_units,number_cycles,days_to_drug_therapy_start,days_to_drug_therapy_end,therapy_types,therapy_type,drug_name,regimen_indication,route_of_administrations,route_of_administration,therapy_ongoing,measure_of_response,radiations,radiation,bcr_radiation_barcode,bcr_radiation_uuid,days_to_radiation_therapy_start,days_to_radiation_therapy_end,radiation_type,radiation_dosage,units,numfractions,anatomic_treatment_site,radiation_treatment_ongoing,course_number
0,96.80.0,BRCA,No,FEMALE,Alive,-19627.0,NaN,1133.0,WHITE,WHITE,E2,No,C50.9,8500/3,C50.9,53,2010.0,NOT HISPANIC OR LATINO,TUMOR FREE,YES,1.0,Allred score 5+3 = 8,Allred score 5+3 = 8,Right Upper Outer Quadrant,Right Upper Outer Quadrant,1.0,Sentinel node biopsy alone,Lumpectomy,NaN,Infiltrating Ductal Carcinoma,Post (prior bilateral ovariectomy OR >12 mo si...,Positive,NO,NaN,NaN,2+,Negative,Tumor resection,Negative,Positive,Equivocal,NaN,0.0,Allred score = 8,Allred score = 8,10-19%,NO,YES,,NaN,NaN,NO,NaN,NaN,7th Stage IIA T2 N0 M0 ...,7th,Stage IIA,T2 N0 M0,T2 N0 M0,T2,N0,M0,YES,YES,NO,NO,9,12,2011,TCGA-E2-A1BD-F19439 BC31751C-7DEF-4DDF-A6D3-...,TCGA-E2-A1BD-F70021 C93AB29A-00FC-49AC-89DC-A...,TCGA-E2-A1BD-F70021,C93AB29A-00FC-49AC-89DC-AC44ED0A1F59,Scheduled Follow-up Submission,NO,,1 TCGA-E2-A1BD-D19445 2534A6C5-8DD5-454D-88...,1 TCGA-E2-A1BD-D19445 2534A6C5-8DD5-454D-888...,NaN,1,TCGA-E2-A1BD-D19445,2534A6C5-8DD5-454D-8887-6098E2B7CBB1,1,mg/day,1,mg/day,NaN,170.0,NaN,Hormone Therapy,Hormone Therapy,Arimidex,ADJUVANT,PO,PO,YES,NaN,TCGA-E2-A1BD-R19442 9FC62023-6140-4514-B9DE-...,TCGA-E2-A1BD-R19442 9FC62023-6140-4514-B9DE-7...,TCGA-E2-A1BD-R19442,9FC62023-6140-4514-B9DE-74BCC48157A0,55.0,10

In [12]:
print(f'Firs cleaned variables: {cleaned_variables}')

Firs cleaned variables: ['tcga_bcr', 'admin', 'bcr', 'file_uuid', 'project_code', 'day_of_dcc_upload', 'month_of_dcc_upload', 'year_of_dcc_upload', 'patient_withdrawal', 'withdrawn', 'program', 'dbgap_registration_code', 'patient', 'additional_studies', 'tumor_tissue_site', 'tumor_tissue_site_other', 'days_to_last_known_alive', 'bcr_patient_barcode', 'patient_id', 'bcr_patient_uuid', 'informed_consent_verified', 'days_to_initial_pathologic_diagnosis', 'axillary_lymph_node_stage_other_method_descriptive_text', 'breast_neoplasm_other_surgical_procedure_descriptive_text', 'breast_carcinoma_primary_surgical_procedure_name', 'histological_type_other', 'breast_cancer_surgery_margin_status', 'init_pathology_dx_method_other', 'pos_finding_her2_erbb2_other_measurement_scale_text', 'her2_erbb_method_calculation_method_text', 'her2_neu_and_centromere_17_copy_number_analysis_input_total_number_count', 'her2_and_centromere_17_positive_finding_other_measurement_scale_text', 'her2_erbb_pos_finding_fl

## 1.2 Minimal explorations

In [14]:
prof = ProfileReport(corrected_clinical_dset, minimal=True)
prof.to_file(output_file='./minimal report corrected clinical data.html')

/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:363: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
IFrame(src='./reports/minimal report corrected clinical data.html', width=1250, height=600)

**Zero quality variables:** 'tcga_bcr', 'admin', 'bcr', 'file_uuid', 'project_code', 'day_of_dcc_upload', 'month_of_dcc_upload', 'year_of_dcc_upload', 'patient_withdrawal', 'withdrawn', 'program', 'dbgap_registration_code', 'patient', 'additional_studies', 'tumor_tissue_site', 'tumor_tissue_site_other', 'days_to_last_known_alive', 'bcr_patient_barcode', 'patient_id', 'bcr_patient_uuid', 'informed_consent_verified', 'days_to_initial_pathologic_diagnosis', 'axillary_lymph_node_stage_other_method_descriptive_text', 'breast_neoplasm_other_surgical_procedure_descriptive_text', 'breast_carcinoma_primary_surgical_procedure_name', 'histological_type_other', 'breast_cancer_surgery_margin_status', 'init_pathology_dx_method_other', 'pos_finding_her2_erbb2_other_measurement_scale_text', 'her2_erbb_method_calculation_method_text', 'her2_neu_and_centromere_17_copy_number_analysis_input_total_number_count', 'her2_and_centromere_17_positive_finding_other_measurement_scale_text', 'her2_erbb_pos_finding_fluorescence_in_situ_hybridization_calculation_method_text', 'fluorescence_in_situ_hybridization_diagnostic_procedure_chromosome_17_signal_result_range', 'metastatic_site_at_diagnosis', 'metastatic_site_at_diagnosis_other', 'metastatic_breast_carcinoma_estrogen_receptor_status', 'metastatic_breast_carcinoma_estrogen_receptor_level_cell_percent_category', 'metastatic_breast_carcinoma_immunohistochemistry_er_pos_cell_score', 'pos_finding_metastatic_breast_carcinoma_estrogen_receptor_other_measuremenet_scale_text', 'metastatic_breast_carcinoma_estrogen_receptor_detection_method_text', 'metastatic_breast_carcinoma_progesterone_receptor_status', 'metastatic_breast_carcinoma_lab_proc_her2_neu_immunohistochemistry_receptor_status', 'metastatic_breast_carcinoma_progesterone_receptor_level_cell_percent_category', 'metastatic_breast_carcinoma_immunohistochemistry_pr_pos_cell_score', 'metastatic_breast_carcinoma_pos_finding_progesterone_receptor_other_measure_scale_text', 'metastatic_breast_carcinoma_progesterone_receptor_detection_method_text', 'metastatic_breast_carcinoma_her2_erbb_pos_finding_cell_percent_category', 'metastatic_breast_carcinoma_erbb2_immunohistochemistry_level_result', 'metastatic_breast_carcinoma_pos_finding_her2_erbb2_other_measure_scale_text', 'metastatic_breast_carcinoma_her2_erbb_method_calculation_method_text', 'metastatic_breast_carcinoma_lab_proc_her2_neu_in_situ_hybridization_outcome_type', 'metastatic_breast_carcinoma_fluorescence_in_situ_hybridization_diagnostic_proc_centromere_17_signal_result_range', 'her2_neu_and_centromere_17_copy_number_metastatic_breast_carcinoma_analysis_input_total_number_count', 'metastatic_breast_carcinoma_her2_neu_chromosone_17_signal_ratio_value', 'metastatic_breast_carcinoma_pos_finding_other_scale_measurement_text', 'metastatic_breast_carcinoma_her2_erbb_pos_finding_fluorescence_in_situ_hybridization_calculation_method_text', 'her2_neu_metastatic_breast_carcinoma_copy_analysis_input_total_number', 'her2_neu_breast_carcinoma_copy_analysis_input_total_number', 'clinical_stage', 'clinical_categories', 'clinical_T', 'clinical_N', 'clinical_M', 'psa', 'psa_value', 'days_to_psa', 'gleason_grading', 'gleason_score', 'primary_pattern', 'secondary_pattern', 'tertiary_pattern', 'ann_arbor', 'b_symptoms', 'extranodal_involvement', 'serum_markers', 'igcccg_stage', 'masaoka_stage', 'days_to_new_tumor_event_after_initial_treatment', 'new_neoplasm_event_type', 'new_neoplasm_event_occurrence_anatomic_site', 'new_neoplasm_occurrence_anatomic_site_text', 'additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'additional_radiation_therapy', 'additional_pharmaceutical_therapy', 'metastatic_breast_carcinoma_immunohistochemistry_er_positive_finding_scale_type', 'metastatic_breast_carcinoma_immunohistochemistry_progesterone_receptor_positive_finding_scale_type', 'new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_additional_surgery_procedure', 'therapy_type_notes', 'clinical_trail_drug_classification', 'regimen_indication_notes', 'radiation_type_notes', 'days_to_additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'additional_study'

**Low quality variables:** 'primary_lymph_node_presentation_assessment', 'cytokeratin_immunohistochemistry_staining_method_micrometastasis_indicator', 'breast_carcinoma_immunohistochemistry_er_pos_finding_scale', 'immunohistochemistry_positive_cell_score', 'her2_immunohistochemistry_level_result', 'breast_carcinoma_progesterone_receptor_status', 'lab_procedure_her2_neu_in_situ_hybrid_outcome_type', 'breast_carcinoma_estrogen_receptor_status', 'lab_proc_her2_neu_immunohistochemistry_receptor_status', 'number_of_lymphnodes_positive_by_ihc', 'first_nonlymph_node_metastasis_anatomic_sites', 'er_level_cell_percentage_category', 'progesterone_receptor_level_cell_percent_category', 'distant_metastasis_present_ind2', 'breast_carcinoma_immunohistochemistry_pos_cell_score', 'lost_follow_up'.

**non informative variables:** 'batch_number', 'disease_code', 'other_dx', 'tissue_source_site', 'icd_o_3_site', 'icd_o_3_histology', 'icd_10', 'year_of_initial_pathologic_diagnosis', 'er_detection_method_text', 'pgr_detection_method_text', 'surgical_procedure_purpose_other_text', 'initial_pathologic_diagnosis_method', 'pos_finding_progesterone_receptor_other_measurement_scale_text', 'positive_finding_estrogen_receptor_other_measurement_scale_text', 'her2_erbb_pos_finding_cell_percent_category', 'breast_carcinoma_immunohistochemistry_progesterone_receptor_pos_finding_scale', 'postoperative_rx_tx', 'follow_ups', 'follow_up', 'year_of_form_completion', 'month_of_form_completion', 'day_of_form_completion', 'bcr_followup_barcode', 'bcr_followup_uuid', 'followup_case_report_form_submission_reason', 'drugs', 'drug', 'tx_on_clinical_trial', 'regimen_number', 'bcr_drug_barcode', 'bcr_drug_uuid', 'total_dose', 'total_dose_units', 'prescribed_dose', 'prescribed_dose_units', 'number_cycles', 'days_to_drug_therapy_start', 'days_to_drug_therapy_end', 'therapy_types', 'therapy_type', 'drug_name', 'regimen_indication', 'route_of_administrations', 'therapy_ongoing', 'measure_of_response', 'radiations', 'radiation', 'bcr_radiation_barcode', 'bcr_radiation_uuid', 'days_to_radiation_therapy_start', 'days_to_radiation_therapy_end', 'radiation_type', 'radiation_dosage', 'units', 'numfractions', 'anatomic_treatment_site', 'radiation_treatment_ongoing', 'course_number', 'route_of_administration', 'system_version','axillary_lymph_node_stage_method_type'.

**redundant vairables:**

* 'race_list', **'race'**.
* 'anatomic_neoplasm_subdivisions', **'anatomic_neoplasm_subdivision'**.
* 'stage_event', **'pathologic_stage'**.
* 'tnm_categories', 'pathologic_categories', **'pathologic_T', 'pathologic_N', 'pathologic_M'**.
* 'new_tumor_events', 'new_tumor_event_after_initial_treatment', 'new_tumor_event'.

**Higly imbalanced variables:** 'gender', 'history_of_neoadjuvant_treatment', 'ethnicity', 'person_neoplasm_cancer_status', 'her2_neu_chromosone_17_signal_ratio_value', 'new_tumor_event_after_initial_treatment'.

**To preprocess:** 'days_to_last_followup', 'lymph_node_examined_count', 'breast_carcinoma_surgical_procedure_name', 'histological_type', 'menopause_status', 'number_of_lymphnodes_positive_by_he', 'pathologic_T', 'pathologic_N', 'pathologic_M', 'pathologic_stage', 'anatomic_neoplasm_subdivision'.

**To usage:** 'vital_status', 'days_to_birth', 'days_to_death', 'age_at_initial_pathologic_diagnosis', 'margin_status', 'tissue_prospective_collection_indicator', 'radiation_therapy','race'.